# Optional Lab - Derivatives
This lab will give you a more intuitive understanding of derivatives. It will show you a simple way of calculating derivatives arithmetically. It will also introduce you to a handy Python library that allows you to calculate derivatives symbolically.

In [16]:
from sympy import symbols, diff, log , Pow, Transpose,MatrixSymbol, exp, Rational 

## Informal definition of derivatives

The formal definition of derivatives can be a bit daunting with limits and values 'going to zero'. The idea is really much simpler. 

The derivative of a function describes how the output of a function changes when there is a small change in an input variable.

Let's use the cost function $J(w)$ as an example. The cost $J$ is the output and $w$ is the input variable.  
Let's give a 'small change' a name *epsilon* or $\epsilon$. We use these Greek letters because it is traditional in mathematics to use *epsilon*($\epsilon$) or *delta* ($\Delta$) to represent a small value. You can think of it as representing 0.001 or some other small value.  

$$
\begin{equation}
\text{if } w \uparrow \epsilon \text{ causes }J(w) \uparrow \text{by }k \times \epsilon \text{ then}  \\
\frac{\partial J(w)}{\partial w} = k \tag{1}
\end{equation}
$$

This just says if you change the input to the function $J(w)$ by a little bit and the output changes by $k$ times that little bit, then the derivative of $J(w)$ is equal to $k$.

Let's try this out.  Let's look at the derivative of the function $J(w) = w^2$ at the point $w=3$ and $\epsilon = 0.001$

In [3]:
J = (3)**2
J_epsilon = (3 + 0.001)**2
k = (J_epsilon - J)/0.001    # difference divided by epsilon
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k:0.6f} ")

J = 9, J_epsilon = 9.006001, dJ_dw ~= k = 6.001000 


We have increased the input value a little bit (0.001), causing the output to change from 9 to 9.006001, an increase of 6 times the input increase. Referencing (1) above, this says that $k=6$, so $\frac{\partial J(w)}{\partial w} \approx 6$. If you are familiar with calculus, you know, written symbolically,  $\frac{\partial J(w)}{\partial w} = 2 w$. With $w=3$ this is 6. Our calculation above is not exactly 6 because to be exactly correct $\epsilon$ would need to be [infinitesimally small](https://www.dictionary.com/browse/infinitesimally) or really, really small. That is why we use the symbols $\approx$ or ~= rather than =. Let's see what happens if we make $\epsilon$ smaller.

In [4]:
J = (3)**2
J_epsilon = (3 + 0.000000001)**2
k = (J_epsilon - J)/0.000000001
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

J = 9, J_epsilon = 9.000000006, dJ_dw ~= k = 6.000000496442226 


The value gets close to exactly 6 as we reduce the size of $\epsilon$. Feel free to try reducing the value further.

## Finding symbolic derivatives
In backprop it is useful to know the derivative of simple functions at any input value. Put another way, we would like to know the 'symbolic' derivative rather than the 'arithmetic' derivative. An example of a symbolic derivative is,  $\frac{\partial J(w)}{\partial w} = 2 w$, the derivative of $J(w) = w^2$ above.  With the symbolic derivative you can find the value of the derivative at any input value $w$.  

If you have taken a calculus course, you are familiar with the many [differentiation rules](https://en.wikipedia.org/wiki/Differentiation_rules#Power_laws,_polynomials,_quotients,_and_reciprocals) that mathematicians have developed to solve for a derivative given an expression. Well, it turns out this process has been automated with symbolic differentiation programs. An example of this in python is the [SymPy](https://www.sympy.org/en/index.html) library. Let's take a look at how to use this.

### $J = w^2$
Define the python variables and their symbolic names.

In [5]:
J, w = symbols('J, w')

Define and print the expression. Note SymPy produces a [latex](https://en.wikibooks.org/wiki/LaTeX/Mathematics) string which generates a nicely readable equation.

In [6]:
J=w**2
J

w**2

Use SymPy's `diff` to differentiate the expression for $J$ with respect to $w$. Note the result matches our earlier example.

In [7]:
dJ_dw = diff(J,w)
dJ_dw

2*w

In [4]:
from sympy import symbols, MatrixSymbol, Transpose, diff, log, exp, Rational

# Declare matrix symbols
W = MatrixSymbol('W', 3, 3)  # Assume 3x3 matrix for demonstration
X = MatrixSymbol('X', 3, 1)  # Assume 3x1 matrix (column vector)
b = MatrixSymbol('b', 3, 1)  # Assume 3x1 matrix (column vector)

# Declare other symbols
y, a, L, Z = symbols('y a L Z')

# Define the loss function L
L = (-y * log(a)) - ((1 - y) * log(1 - a))

# Differentiate L with respect to a
dL_da = diff(L, a)

# Define the activation function a
a_expr = Rational(1, 1) / (1 + exp(-Z))

# Differentiate a with respect to Z
da_dZ = diff(a_expr, Z)

# Using chain rule, dL/dZ = dL/da * da/dZ
dL_dZ = dL_da * da_dZ

print("dL/da =", dL_da)
print("dL/dZ =", dL_dZ)


dL/da = (1 - y)/(1 - a) - y/a
dL/dZ = ((1 - y)/(1 - a) - y/a)*exp(-Z)/(1 + exp(-Z))**2


In [10]:
from sympy import symbols, diff, log, exp, Matrix

# Define symbols
w1, w2, w3 = symbols('w1 w2 w3')
x1, x2, x3 = symbols('x1 x2 x3')
b, y, a, Z, L = symbols('b y a Z L')

# Form the weight and data vectors
w = Matrix([w1, w2, w3])
X = Matrix([x1, x2, x3])

# Define equations
Z = w.dot(X) + b  # Z = w^T * X + b
a = 1 / (1 + exp(-Z))  # Sigmoid function
L = -y * log(a) - (1 - y) * log(1 - a)  # Binary Cross-Entropy Loss

# Differentiate L with respect to a
dL_da = diff(L, a)

# Differentiate L with respect to Z
dL_dZ = diff(L.subs(a, 1 / (1 + exp(-Z))), Z)

# Differentiate Z with respect to w and b
dZ_dw = [diff(Z, wi) for wi in [w1, w2, w3]]

# Gradient with respect to w and b
dw = [dwi * dL_dZ for dwi in dZ_dw]  # Chain rule applied to each element
db = diff(Z, b) * dL_dZ  # Chain rule

print(f'dw: {dw}')
print(f'db: {db}')


ValueError: 
Can't calculate derivative wrt 1/(exp(-b - w1*x1 - w2*x2 - w3*x3) +
1).

In [4]:
from sympy import symbols, diff, exp, log, simplify

# Declare symbols
y, a, Z, w1, w2, w3, x1, x2, x3, b = symbols('y a Z w1 w2 w3 x1 x2 x3 b')

# Define Loss Function (Cross-Entropy)
L = -y * log(a) - (1 - y) * log(1 - a)

# Define Z in terms of w1, w2, w3, x1, x2, x3, and b
Z = w1 * x1 + w2 * x2 + w3 * x3 + b

# Define a in terms of Z (Sigmoid function)
a = 1 / (1 + exp(-Z))

# Differentiate L with respect to a
dL_da = diff(L, a)

# Differentiate a in terms of Z (Derivative of Sigmoid)
da_dZ = diff(a, Z)

# Compute dL/dZ using chain rule
dL_dZ = simplify(dL_da * da_dZ)

# Differentiate Z with respect to w1, w2, w3, and b
dZ_dw1 = diff(Z, w1)
dZ_dw2 = diff(Z, w2)
dZ_dw3 = diff(Z, w3)
dZ_db = diff(Z, b)

# Compute gradient with respect to w1, w2, w3, and b using chain rule
dw1 = dZ_dw1 * dL_dZ
dw2 = dZ_dw2 * dL_dZ
dw3 = dZ_dw3 * dL_dZ
db = dZ_db * dL_dZ

print("dL/dZ:", dL_dZ)
print("dw1:", dw1)
print("dw2:", dw2)
print("dw3:", dw3)
print("db:", db)


ValueError: 
Can't calculate derivative wrt 1/(exp(-b - w1*x1 - w2*x2 - w3*x3) +
1).

In [5]:
import numpy as np

# Assuming we have some data for X, y, w, and b
X = np.array([[0.1, 0.2], [0.4, 0.5], [0.5, 0.3]])  # shape (3, 2)
y = np.array([0, 1, 0])  # shape (3,)
w = np.array([0.1, 0.2])  # shape (2,)
b = 0.1  # scalar

# Forward pass to compute Z and a
Z = np.dot(X, w) + b  # shape (3,)
a = 1 / (1 + np.exp(-Z))  # shape (3,)

# Compute derivatives
dL_dZ = a - y  # shape (3,)
dZ_dw = X  # shape (3, 2)
dZ_db = 1  # scalar

# Compute gradient with respect to w and b
dL_dw = np.dot(dL_dZ, dZ_dw)  # shape (2,)
dL_db = np.sum(dL_dZ * dZ_db)  # scalar

print("Gradient with respect to w:", dL_dw)
print("Gradient with respect to b:", dL_db)


Gradient with respect to w: [0.1537824  0.05303517]
Gradient with respect to b: 0.6494514041852678


In [8]:
from sympy import symbols, diff, log, exp

# Define symbols
w1, x1, b, y = symbols('w1 x1 b y')
Z = symbols('Z')

# Define equations
a = 1 / (1 + exp(-Z))  # Sigmoid function
L = -y * log(a) - (1 - y) * log(1 - a)  # Binary Cross-Entropy Loss

# Substitute the sigmoid function into the loss function
L_sub = L.subs(a, 1 / (1 + exp(-Z)))

# Differentiate L with respect to Z
dL_dZ = diff(L_sub, Z)

# Differentiate Z with respect to w1 and b
dZ_dw1 = diff(Z, w1)
dZ_db = diff(Z, b)
print(dZ_dw1)
# Compute gradient with respect to w1 and b using the chain rule
dL_dw1 = dL_dZ * dZ_dw1
dL_db = dL_dZ * dZ_db

print("dL/dZ =", dL_dZ)
print("dL/dw1 =", dL_dw1)
print("dL/db =", dL_db)


0
dL/dZ = -y*exp(-Z)/(1 + exp(-Z)) + (1 - y)*exp(-Z)/((1 - 1/(1 + exp(-Z)))*(1 + exp(-Z))**2)
dL/dw1 = 0
dL/db = 0


In [10]:
from sympy import symbols, diff, log, exp, simplify

w1, x1, b, y, a, Z = symbols('w1 x1 b y a Z')


Z = w1 * x1 + b
a = 1 / (1 + exp(-Z))
L = -y * log(a) - (1 - y) * log(1 - a)


# Differentiate L with respect to a
dL_da = diff(L, a)

# Differentiate L with respect to Z
dL_dZ = diff(L.subs(a, 1 / (1 + exp(-Z))), Z)

# Differentiate Z with respect to w1 and b
dZ_dw1 = diff(Z, w1)
dZ_db = diff(Z, b)

# Compute gradient with respect to w1 and b using the chain rule
dL_dw1 = simplify(dL_dZ * dZ_dw1)
dL_db = simplify(dL_dZ * dZ_db)

print("dL/dZ =", dL_dZ)
print("dL/dw1 =", dL_dw1)
print("dL/db =", dL_db)


ValueError: 
Can't calculate derivative wrt 1/(exp(-b - w1*x1) + 1).

Evaluate the derivative at a few points by 'substituting' numeric values for the symbolic values. In the first example, $w$ is replaced by $2$.

In [8]:
dJ_dw.subs([(w,2)])    # derivative at the point w = 2

4

In [9]:
dJ_dw.subs([(w,3)])    # derivative at the point w = 3

6

In [10]:
dJ_dw.subs([(w,-3)])    # derivative at the point w = -3

-6

## $J = 2w$

In [11]:
w, J = symbols('w, J')

In [12]:
J = 2 * w
J

2*w

In [13]:
dJ_dw = diff(J,w)
dJ_dw

2

In [15]:
dJ_dw.subs([(w,-3)])    # derivative at the point w = -3

2

Compare this with the arithmetic calculation

In [16]:
J = 2*3
J_epsilon = 2*(3 + 0.001)
k = (J_epsilon - J)/0.001
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

J = 6, J_epsilon = 6.002, dJ_dw ~= k = 1.9999999999997797 


For the function $J=2w$, it is easy to see that any change in $w$ will result in 2 times that amount of change in the output $J$, regardless of the starting value of $w$. Our NumPy and arithmetic results confirm this. 

## $J = w^3$

In [17]:
J, w = symbols('J, w')

In [18]:
J=w**3
J

w**3

In [19]:
dJ_dw = diff(J,w)
dJ_dw

3*w**2

In [20]:
dJ_dw.subs([(w,2)])   # derivative at the point w=2

12

Compare this with the arithmetic calculation

In [21]:
J = (2)**3
J_epsilon = (2+0.001)**3
k = (J_epsilon - J)/0.001
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

J = 8, J_epsilon = 8.012006000999998, dJ_dw ~= k = 12.006000999997823 


## $J = \frac{1}{w}$

In [ ]:
J, w = symbols('J, w')

In [23]:
J= 1/w
J

1/w

In [24]:
dJ_dw = diff(J,w)
dJ_dw

-1/w**2

In [25]:
dJ_dw.subs([(w,2)])

-1/4

Compare this with the arithmetic calculation

In [ ]:
J = 1/2
J_epsilon = 1/(2+0.001)
k = (J_epsilon - J)/0.001
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

## $J = \frac{1}{w^2}$

In [27]:
J, w = symbols('J, w')

If you have time, try to repeat the above steps on the function  $J = \frac{1}{w^2}$ and evaluate at w=4

Compare this with the arithmetic calculation

In [ ]:
J = 1/4**2
J_epsilon = 1/(4+0.001)**2
k = (J_epsilon - J)/0.001
print(f"J = {J}, J_epsilon = {J_epsilon}, dJ_dw ~= k = {k} ")

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
```python 
J= 1/w**2
dJ_dw = diff(J,w)
dJ_dw.subs([(w,4)])
```
  

</details>

    


## Congratulations!
If you have run through the above examples, you understand a derivative describes the change in the output of a function that is a result of a small change in an input to that function. You also can use *SymPy* in python to find the symbolic derivative of functions.